In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns',25)

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,learning_curve,GridSearchCV
from sklearn.decomposition import PCA
from collections import Counter

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,RidgeClassifier,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier




In [ ]:
df = pd.read_csv('../input/performance-prediction/summary.csv')
df.head()

# Prepare Data

In [ ]:
df1 = df.copy()

In [ ]:
na_cols = ['3PointPercent']
df1.loc[:,na_cols] = df1.loc[:,na_cols].fillna(np.mean(df1.loc[:,na_cols]))

# Up Sampling

In [ ]:
plt.figure(figsize=(8,8))
his = sns.histplot(data=df1,x='Target',hue='Target',bins=2)
locs = [(bar.get_width(),bar.get_height()) for bar in his.patches]
count = df['Target'].groupby(by=df['Target']).count()
plt.text(locs[2][0]/2,locs[2][1]+3,count[0],size=12)
plt.text(locs[1][0]*1.5,locs[1][1]+3,count[1],size=12)
plt.show()

In [ ]:
target = df1.loc[:,'Target']
train = df1.drop(columns=['Target','Name'],axis=1)

In [ ]:
from imblearn.combine import SMOTETomek

sample = SMOTETomek(random_state=12)
X_sample,y_sample = sample.fit_resample(train,target)

In [ ]:
Counter(y_sample)

# Split Data

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_sample,y_sample,test_size=0.33,random_state=42)

# Scale Data

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training

In [ ]:
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'RidgeClassifier': RidgeClassifier(),
    'SGDClassifier': SGDClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier()
}

In [ ]:
for name,model in models.items():
    print(f"{name} Acc score: {np.mean(cross_val_score(model,X_train,y_train,cv=5))}")

In [ ]:
base_model1 = RandomForestClassifier()
base_model1.fit(X_train,y_train)
print(f"Accuracy = {round(base_model1.score(X_test,y_test),2)*100}%")

In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(RandomForestClassifier(),
X_train,y_train, train_sizes=[20,50, 80, 110,200,300,354,500,600,717], cv=5,random_state=42)
ax = sns.lineplot(x=train_sizes,y=np.mean(train_scores,axis=1),marker='o',)
ax.annotate('train scores',(300,0.76),color='g',size=11)
sns.lineplot(x=train_sizes,y=np.mean(valid_scores,axis=1),marker='o')
ax.annotate('val scores',(300,0.66),color='r',size=11)
plt.title('Learning Curve')
plt.show()